In [ ]:
import numpy as np
import pandas as pd
import wave
import librosa
import re
from tensorflow_hub import load, Module
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input, LSTM, Bidirectional
from tensorflow.keras.models import Model

In [ ]:
!pip install contractions

In [ ]:
import contractions

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/data_ML5/

Mounted at /content/drive
/content/drive/MyDrive/data_ML5


In [ ]:
elmo = load("https://tfhub.dev/google/elmo/3").signatures["default"]

In [ ]:
train_split_df = pd.read_csv('train_split_Depression_AVEC2017.csv')
test_split_df = pd.read_csv('dev_split_Depression_AVEC2017.csv')
train_split_num = train_split_df[['Participant_ID']]['Participant_ID'].tolist()
test_split_num = test_split_df[['Participant_ID']]['Participant_ID'].tolist()
# train_split_clabel = train_split_df[['PHQ8_Binary']]['PHQ8_Binary'].tolist()
# test_split_clabel = test_split_df[['PHQ8_Binary']]['PHQ8_Binary'].tolist()

train_split_clabel = train_split_df[['PHQ8_Score']]['PHQ8_Score'].tolist()
test_split_clabel = test_split_df[['PHQ8_Score']]['PHQ8_Score'].tolist()


topics = []
with open('questions.txt', 'r') as f:
    for line in f.readlines():
        topics.append(line.strip('\n').strip())

In [ ]:
print("Count 0: ", train_split_clabel.count(0))
print("Count 1: ", train_split_clabel.count(1))

Count 0:  77
Count 1:  30


In [ ]:
def identify_topics(sentence):
    sentence = re.sub(r'\(|\)', '', sentence)
    pattern = r'\b(what|how|where|when|why|are|do|have|who|who\'s|what\'s|why\'d|what\'d)\b(.*)$'
    match = re.search(pattern, sentence, re.IGNORECASE)
    if match:
        question = match.group(0).strip()
        if question in topics:
          return True
    return False

def expanding_contractions(text):
  # expanding contractions
  expanded_words = []
  for word in text.split():
  # using contractions.fix to expand the shortened words
    expanded_words.append(contractions.fix(word))

  text = ' '.join(expanded_words)
  return text



def extract_features(number, text_features, target, mode, text_targets):
    print(number)
    transcript = pd.read_csv('{0}_TRANSCRIPT.csv'.format(number), sep='\t').fillna('')

    time_range = []
    responses = []
    response = ''
    response_flag = False


    for t in transcript.itertuples():
        if getattr(t,'speaker') == 'Ellie':
            content = getattr(t,'value').strip()
            if identify_topics(content):
                response_flag = True
                if len(response) != 0:
                    response = expanding_contractions(response.strip())
                    response = re.sub(r'[^\w\s]', '', response)
                    responses.append(response)
                response = ''
            elif response_flag and len(content.split()) > 4:
                response_flag = False
                if len(response) != 0:
                    response = expanding_contractions(response)
                    response = re.sub(r'[^\w\s]', '', response)
                    responses.append(response)
                response = ''
        elif getattr(t,'speaker') == 'Participant':
            if 'scrubbed_entry' in getattr(t,'value'):
                continue
            elif response_flag:
                response +=' ' +getattr(t,'value').split('\n')[0].strip()

    if len(responses) == 0:
      print("Empty")
    else:
      text_feature = elmo(tf.constant(responses))["elmo"]
      text_feature = tf.reduce_mean(text_feature, axis=1)

      print(text_feature.shape)

      text_features.append(text_feature)
      text_targets.append(target)




In [ ]:
# training set
text_features_train = []
text_ctargets_train = []

# test set
text_features_test = []
text_ctargets_test = []


In [ ]:
for index in range(len(train_split_num)):
     extract_features(train_split_num[index], text_features_train, train_split_clabel[index], 'train', text_ctargets_train)


303
(33, 1024)
304
(40, 1024)
305
(41, 1024)
310
(40, 1024)
312
(39, 1024)
313
(36, 1024)
315
(44, 1024)
316
(48, 1024)
317
(42, 1024)
318
(33, 1024)
319
(38, 1024)
320
(53, 1024)
321
(54, 1024)
322
(48, 1024)
324
(39, 1024)
325
(37, 1024)
326
(44, 1024)
327
(46, 1024)
328
(38, 1024)
330
(41, 1024)
333
(42, 1024)
336
(46, 1024)
338
(37, 1024)
339
(44, 1024)
340
(41, 1024)
341
(45, 1024)
343
(47, 1024)
344
(45, 1024)
345
(33, 1024)
347
(43, 1024)
348
(48, 1024)
350
(40, 1024)
351
(48, 1024)
352
(31, 1024)
353
(39, 1024)
355
(34, 1024)
356
(34, 1024)
357
(29, 1024)
358
(40, 1024)
360
(32, 1024)
362
(41, 1024)
363
(27, 1024)
364
(31, 1024)
366
(26, 1024)
368
(23, 1024)
369
(28, 1024)
370
(21, 1024)
371
(38, 1024)
372
(34, 1024)
374
(33, 1024)
375
(33, 1024)
376
(31, 1024)
379
(27, 1024)
380
(31, 1024)
383
(27, 1024)
385
(34, 1024)
386
(34, 1024)
391
(31, 1024)
392
(32, 1024)
393
(26, 1024)
397
(37, 1024)
400
(38, 1024)
401
(31, 1024)
402
(30, 1024)
409
(22, 1024)
412
(37, 1024)
414
(26, 1

In [ ]:
for index in range(len(test_split_num)):
  extract_features(test_split_num[index], text_features_test, test_split_clabel[index], 'test', text_ctargets_test)

302
(31, 1024)
307
(30, 1024)
331
(39, 1024)
335
(34, 1024)
346
(39, 1024)
367
(29, 1024)
377
(36, 1024)
381
(35, 1024)
382
(38, 1024)
388
(34, 1024)
389
(45, 1024)
390
(34, 1024)
395
(33, 1024)
403
(28, 1024)
404
(30, 1024)
406
(26, 1024)
413
(31, 1024)
417
(27, 1024)
418
(34, 1024)
420
(39, 1024)
422
(25, 1024)
436
(34, 1024)
439
(26, 1024)
440
(37, 1024)
451
Empty
458
Empty
472
(31, 1024)
476
(31, 1024)
477
(32, 1024)
482
(31, 1024)
483
(32, 1024)
484
(32, 1024)
489
(36, 1024)
490
(38, 1024)
492
(32, 1024)


In [ ]:
# data imbalance
X_train = []
Y_train = []
X_test = []
Y_test = []

counter = 0

cut = 10
debt = 0

for i in range(len(text_features_train)):
  # regression
    if text_ctargets_train[i] >= 10:
        times = 3 if counter < 48 else 2
        for j in range(times):
            if (j+1)*cut > len(text_features_train[i]):
                debt+=1
                continue
            X_train.append(text_features_train[i][j*cut:(j+1)*cut])
            Y_train.append(text_ctargets_train[i])
            if debt > 0:
                debt -= 1
            counter+=1
    else:
        X_train.append(text_features_train[i][:cut])
        Y_train.append(text_ctargets_train[i])


for i in range(len(text_features_test)):
    X_test.append(text_features_test[i][:cut])
    Y_test.append(text_ctargets_test[i])

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(154, 10, 1024)
(154,)
(33, 10, 1024)
(33,)


In [ ]:
np.savez('text_train_X_reg_new1.npz', X_train)
np.savez('text_train_Y_reg_new1.npz', Y_train)
np.savez('text_test_X_reg_new1.npz', X_test)
np.savez('text_test_Y_reg_new1.npz', Y_test)

In [ ]:
# data imbalance
X_train = []
Y_train = []
X_test = []
Y_test = []

counter = 0

cut = 10
debt = 0

for i in range(len(text_features_train)):
  # classification
    if text_ctargets_train[i] == 1:
        times = 3+debt if counter < 48 else 2+debt
        for j in range(times):
            if (j+1)*cut > len(text_features_train[i]):
                debt+=1
                continue
            X_train.append(text_features_train[i][j*cut:(j+1)*cut])
            Y_train.append(text_ctargets_train[i])
            if debt > 0:
                debt -= 1
            counter+=1
    else:
        X_train.append(text_features_train[i][:cut])
        Y_train.append(text_ctargets_train[i])


for i in range(len(text_features_test)):
    X_test.append(text_features_test[i][:cut])
    Y_test.append(text_ctargets_test[i])

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [ ]:
print("Count 0: ", list(Y_train).count(0))
print("Count 1: ", list(Y_train).count(1))

Count 0:  77
Count 1:  76


In [ ]:
np.savez('text_train_X_cla1.npz', X_train)
np.savez('text_train_Y_cla1.npz', Y_train)
np.savez('text_test_X_cla1.npz', X_test)
np.savez('text_test_Y_cla1.npz', Y_test)

# Classification

In [ ]:
X_train = np.load('text_train_X_cla1.npz', allow_pickle=True)['arr_0']
Y_train = np.load('text_train_Y_cla1.npz', allow_pickle=True)['arr_0']
X_test = np.load('text_test_X_cla1.npz', allow_pickle=True)['arr_0']
Y_test = np.load('text_test_Y_cla1.npz', allow_pickle=True)['arr_0']


In [ ]:
train_y = to_categorical(Y_train)
test_y = to_categorical(Y_test)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(153, 10, 1024)
(153,)
(33, 10, 1024)
(33,)


In [ ]:
class AttentionLayer_1(tf.keras.Model):
    def __init__(self, hidden_dims):
        super(AttentionLayer_1, self).__init__()

        self.linear_layer = tf.keras.layers.Dense(hidden_dims)
        self.relu_layer = tf.keras.layers.ReLU()

    def call(self, inputs):
        x = self.linear_layer(inputs)
        x = self.relu_layer(x)
        return x


def attention(output, final_hidden_state):
    lstm_hidden = tf.expand_dims(tf.convert_to_tensor(final_hidden_state), 1)
    attention_layer = AttentionLayer_1(hidden_dims=128)
    atten_w = attention_layer(lstm_hidden)
    tanh = tf.keras.activations.tanh(output)
    c = tf.matmul(atten_w, tf.transpose(tanh, (0, 2, 1)))
    softmax_w = tf.keras.activations.softmax(c, axis=-1)
    context = tf.matmul(softmax_w, output)
    result = tf.squeeze(context, axis=1)
    return result

def create_model():

    inputs = Input(shape=(10, 1024))

    forward_layer = LSTM(128,  activation='relu', dropout=0.5, return_sequences=True, return_state=True)
    backward_layer = LSTM(128, activation='relu',  dropout=0.5, return_sequences=True, go_backwards=True, return_state=True)
    bidir = Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(10, 1024), merge_mode='sum')

    output, forward_state, forward_sell_tate, backward_state, backward_sell_state = bidir(inputs)
    final_hidden_state = forward_state + backward_state

    x = attention(output, final_hidden_state)

    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)

    outputs = Dense(2, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model



In [ ]:
model = create_model()
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001), metrics=['accuracy'])

In [ ]:
model.fit(X_train, train_y, epochs=30, validation_data=(X_test, test_y), batch_size=8)

Epoch 1/30
20/20 [==============================] - 7s 85ms/step - loss: 0.6949 - accuracy: 0.5163 - val_loss: 0.6928 - val_accuracy: 0.4848
Epoch 2/30
20/20 [==============================] - 2s 86ms/step - loss: 0.6863 - accuracy: 0.5621 - val_loss: 0.6886 - val_accuracy: 0.5152
Epoch 3/30
20/20 [==============================] - 2s 85ms/step - loss: 0.6855 - accuracy: 0.6013 - val_loss: 0.6844 - val_accuracy: 0.5455
Epoch 4/30
20/20 [==============================] - 2s 83ms/step - loss: 0.6714 - accuracy: 0.5817 - val_loss: 0.6819 - val_accuracy: 0.5152
Epoch 5/30
20/20 [==============================] - 1s 45ms/step - loss: 0.6652 - accuracy: 0.5817 - val_loss: 0.6746 - val_accuracy: 0.5758
Epoch 6/30
20/20 [==============================] - 1s 44ms/step - loss: 0.6484 - accuracy: 0.6471 - val_loss: 0.6584 - val_accuracy: 0.6364
Epoch 7/30
20/20 [==============================] - 1s 45ms/step - loss: 0.6568 - accuracy: 0.6144 - val_loss: 0.6392 - val_accuracy: 0.6364
Epoch 8/30
20

In [ ]:
y_pred = model.predict(X_test)

2/2 [==============================] - 0s 13ms/step


In [ ]:
predicted_1 = [1 if x[1] > x[0] else 0 for x in y_pred]
print(classification_report(Y_test, predicted_1))

              precision    recall  f1-score   support

           0       0.74      0.95      0.83        21
           1       0.83      0.42      0.56        12

    accuracy                           0.76        33
   macro avg       0.79      0.68      0.69        33
weighted avg       0.77      0.76      0.73        33



In [ ]:
model.save("model_text_class1.keras")

# Regression

In [ ]:
X_train = np.load('text_train_X_reg_new1.npz', allow_pickle=True)['arr_0']
Y_train = np.load('text_train_Y_reg_new1.npz', allow_pickle=True)['arr_0']
X_test = np.load('text_test_X_reg_new1.npz', allow_pickle=True)['arr_0']
Y_test = np.load('text_test_Y_reg_new1.npz', allow_pickle=True)['arr_0']


In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(154, 10, 1024)
(154,)
(33, 10, 1024)
(33,)


In [ ]:
class AttentionLayer_1(tf.keras.Model):
    def __init__(self, hidden_dims):
        super(AttentionLayer_1, self).__init__()

        self.linear_layer = tf.keras.layers.Dense(hidden_dims)
        self.relu_layer = tf.keras.layers.ReLU()

    def call(self, inputs):
        x = self.linear_layer(inputs)
        x = self.relu_layer(x)
        return x


def attention(output, final_hidden_state):
    lstm_hidden = tf.expand_dims(tf.convert_to_tensor(final_hidden_state), 1)
    attention_layer = AttentionLayer_1(hidden_dims=128)
    atten_w = attention_layer(lstm_hidden)
    tanh = tf.keras.activations.tanh(output)
    c = tf.matmul(atten_w, tf.transpose(tanh, (0, 2, 1)))
    softmax_w = tf.keras.activations.softmax(c, axis=-1)
    context = tf.matmul(softmax_w, output)
    result = tf.squeeze(context, axis=1)
    return result

def create_model():

    inputs = Input(shape=(10, 1024))

    forward_layer = LSTM(128,  activation='relu', dropout=0.5, return_sequences=True, return_state=True)
    backward_layer = LSTM(128, activation='relu',  dropout=0.5, return_sequences=True, go_backwards=True, return_state=True)
    bidir = Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(10, 1024), merge_mode='sum')

    output, forward_state, forward_sell_tate, backward_state, backward_sell_state = bidir(inputs)
    final_hidden_state = forward_state + backward_state

    x = attention(output, final_hidden_state)

    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)

    outputs = Dense(1, activation='linear')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model


In [ ]:
model = create_model()
model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit(X_train, Y_train, epochs=50, validation_data=(X_test, Y_test), batch_size=8)

Epoch 1/50
20/20 [==============================] - 6s 68ms/step - loss: 106.8831 - mean_absolute_error: 8.5301 - val_loss: 101.1672 - val_mean_absolute_error: 7.5665
Epoch 2/50
20/20 [==============================] - 1s 44ms/step - loss: 106.3080 - mean_absolute_error: 8.5020 - val_loss: 100.7068 - val_mean_absolute_error: 7.5460
Epoch 3/50
20/20 [==============================] - 1s 45ms/step - loss: 105.9459 - mean_absolute_error: 8.4855 - val_loss: 100.2379 - val_mean_absolute_error: 7.5255
Epoch 4/50
20/20 [==============================] - 1s 46ms/step - loss: 105.1837 - mean_absolute_error: 8.4514 - val_loss: 99.7719 - val_mean_absolute_error: 7.5050
Epoch 5/50
20/20 [==============================] - 1s 41ms/step - loss: 104.4919 - mean_absolute_error: 8.4116 - val_loss: 99.2626 - val_mean_absolute_error: 7.4826
Epoch 6/50
20/20 [==============================] - 1s 44ms/step - loss: 103.8253 - mean_absolute_error: 8.3958 - val_loss: 98.7080 - val_mean_absolute_error: 7.4576
E

In [ ]:
loss, mean_absolute_error = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}, Test Mean Absolute Error: {mean_absolute_error}")

2/2 [==============================] - 0s 13ms/step - loss: 35.8976 - mean_absolute_error: 5.2236
Test Loss: 35.897621154785156, Test Mean Absolute Error: 5.223595142364502


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(np.sqrt(mean_squared_error(Y_test,y_pred)))

5.991462230676506


In [ ]:
model.save("model_text_regression1.keras")